<a href="https://colab.research.google.com/github/ronitagarwala01/MED_263_Group_1/blob/main/MED263_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview


#Step 1: Install and Import Libraries

In [ ]:

# @title Install Dependencies: takes 1-2 minutes to run
!pip install transformers datasets evaluate accelerate pillow torchvision scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import torch
import torchvision
import numpy as np
import evaluate
from datasets import load_dataset
from huggingface_hub import notebook_login
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import DefaultDataCollator
from transformers import AutoImageProcessor
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
import matplotlib.pyplot as plt

# Step 2: Load Dataset

In [ ]:
# @title Download data from github

! git clone https://github.com/allbert4/MED_263_Group_1

In [ ]:
# @title Unpackage image data into Colab file system

import zipfile
import os

# Define paths
zip_path = "/content/MED_263_Group_1/HAM10000_sampled_100_balance.zip"  # TODO
extract_path = "HAM10000_images"

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Images extracted to:", extract_path)


In [ ]:
# @title Load labels from Github data labels file
import pandas as pd

labels_path = "/content/MED_263_Group_1/HAM10000_sampled_100bal_metadata.csv"
df = pd.read_csv(labels_path)

# Display the first few rows
print(df.head())


In [ ]:
# @title Convert data to Huggingface dataset
from datasets import Dataset

base_path = extract_path + "/HAM10000_sampled_100_balance/"
dataset_dict = {
    "image": [os.path.join(base_path, img + ".jpg") for img in df["image_id"]],
    "label": df["dx"].tolist()
}

num_classes = len(set(df["dx"]))

dataset = Dataset.from_dict(dataset_dict)
print(dataset)

In [ ]:
# @title Check for missing images- should be 0
missing_files = [img for img in dataset["image"] if not os.path.exists(img)]
print(f"Missing images: {len(missing_files)}")
assert len(missing_files) == 0, "Files not found, check your file directory paths"

In [ ]:
# Problem: labels are strings. We need to convert them to integers
# show example
dataset[0]['label']

In [ ]:
# @title Assign indices to class labels

unique_labels = sorted(set(dataset["label"]))  # Ensure consistent ordering
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

print("Label Mapping:", label2id)

In [ ]:
# @title Convert labels to integers

def convert_labels(example):
    example["label"] = label2id[example["label"]]
    return example

dataset = dataset.map(convert_labels)

In [ ]:
# Now labels are integers!
# Show example
dataset[0]['label']

In [ ]:
# Convert to Image Dataset
from datasets import Image

dataset = dataset.cast_column("image", Image())

In [ ]:
# @title Train/test data split
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

#Step 3: Data Visualization and Statistics

In [ ]:
print(train_dataset[0])

In [ ]:
# @title Visualize a few example images from each class
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
images = []
for label in range(num_classes):
    for example in train_dataset:
      if example["label"] == label:
          image=example["image"]
          images.append(image)
          break

for cls, img in enumerate(images):
  plt.subplot(3, 3, cls+1)
  plt.imshow(images[cls], cmap="gray")
  plt.title(f"Class {cls}: {id2label[cls]}")

plt.tight_layout(pad=3.0)
plt.show()

## Next we visualize the distribution of our dataset.

In [ ]:
labs = [example["label"] for example in train_dataset]
plt.hist(labs, bins=num_classes)

# Add title and labels
plt.title('Distribution of Train Set Classes')
plt.xlabel('Class')
plt.ylabel('Frequency')

# Show the plot
plt.show()

labs = [example["label"] for example in test_dataset]
plt.hist(labs, bins=num_classes)

# Add title and labels
plt.title('Distribution of Test Set Classes')
plt.xlabel('Class')
plt.ylabel('Frequency')

# Show the plot
plt.show()


In [ ]:
# Visualize training data statistics

# Label Statistics
import seaborn as sns

dx_col = df['dx']
sns.countplot(x=dx_col, data=df)
plt.title('Distribution of Class Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize Distribution for Diagnoses Type

dx_type_col = df['dx_type']
sns.countplot(x=dx_type_col, data=df)
plt.title('Diagnoses Types')
plt.xlabel('Diagnosis Type')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize Distribution of Age of Patients

age_col = df['age']
sns.countplot(x=age_col, data=df)
plt.title('Age of Patients')
plt.xlabel('Age')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize Distribution of Sex of Patients

sex_col = df['sex']
sns.countplot(x=sex_col, data=df)
plt.title('Sex of Patients')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize Distribution of Localization

localization_col = df['localization']
sns.countplot(x=localization_col, data=df)
plt.title('Localization of Patches')
plt.xlabel('Localization')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

#Step 4: Load Models

In [ ]:
# @title Load base model and image processor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
# @title Do some image transformations to make the model more resistant to overfitting, and crop to correct size for model
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)

_transforms = Compose([
    RandomResizedCrop(size),
    ToTensor(),  # Convert to tensor
    normalize  # Apply normalization
])

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples


In [ ]:
# @title Preprocess over entire dataset
train_dataset = train_dataset.with_transform(transforms)
test_dataset = test_dataset.with_transform(transforms)

In [ ]:
# @title Batch examples
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [ ]:
# @title Set up evaluation metrics
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

#Step 5: Fine tune models

In [ ]:

# @title Set up model
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=num_classes
)

In [ ]:
# @title Hyperparameters
training_args = TrainingArguments(
    output_dir="tuned_model",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,                     # Learning rate
    lr_scheduler_type="cosine",
    per_device_train_batch_size=16,         # Batch size train
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,          # Batch size eval
    num_train_epochs=30,                     # Number of epochs
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    processing_class=image_processor
)

trainer.train()

#Step 6: Evaluate Performance
todo:
-  Evaluate model performance on validation set using:
  - Accuracy (using `evaluate.load("accuracy")`)
  - Precision, Recall, F1-score (using `precision_recall_fscore_support`)
  - View the trend of loss and accuracy curve
  - Confusion Matrix (for error analysis)


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

predictions_output = trainer.predict(test_dataset)

predicted_labels = np.argmax(predictions_output.predictions, axis=1)

true_labels = predictions_output.label_ids

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    if predictions.ndim > 1:
        predictions = np.argmax(predictions, axis=1)

    print(f"Predictions shape: {predictions.shape}, Labels shape: {labels.shape}")  # Debugging step

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=1)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1
    }

metrics = compute_metrics((predicted_labels, true_labels))
print(metrics)

In [ ]:
for log in trainer.state.log_history:
    print(log) #Check whether loss decreases

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

for epoch in range(1, int(trainer.state.epoch) + 1):
    train_predictions = trainer.predict(train_dataset)
    train_preds = train_predictions.predictions.argmax(axis=1)
    train_labels = train_predictions.label_ids
    train_accuracy = accuracy_score(train_labels, train_preds)

    trainer.state.log_history.append({"epoch": epoch, "train_accuracy": train_accuracy})

metrics = trainer.state.log_history

train_loss = [entry["loss"] for entry in metrics if "loss" in entry]
test_loss = [entry["eval_loss"] for entry in metrics if "eval_loss" in entry]

train_accuracy = [entry["train_accuracy"] for entry in metrics if "train_accuracy" in entry]
test_accuracy = [entry["eval_accuracy"] for entry in metrics if "eval_accuracy" in entry]


plt.figure(figsize=(13,4))

# Loss Curve
plt.subplot(1,2,1)
plt.title("Loss")
plt.plot(train_loss, label="Train")
plt.plot(test_loss, label="Test")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid(True)
plt.legend(loc="best")

# Accuracy Curve
plt.subplot(1,2,2)
plt.title("Accuracy")
plt.plot(train_accuracy, label="Train")
plt.plot(test_accuracy, label="Test")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.grid(True)
plt.legend(loc="best")

plt.show()

In [ ]:
print("Unique values in true_labels:", np.unique(true_labels))
print("Class distribution in true_labels:", np.bincount(true_labels))


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

predictions_output = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions_output.predictions, axis=1)
true_labels = predictions_output.label_ids

report = classification_report(true_labels, predicted_labels)
print(report)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(true_labels, predicted_labels)
print(cm)

all_labels = np.unique(np.concatenate((true_labels, predicted_labels)))
plt.figure(figsize=(6,6))
plt.title("Confusion Matrix Heatmap", fontsize=13)
sns.heatmap(cm, cmap="Reds", annot=True, fmt="d", xticklabels=all_labels, yticklabels=all_labels)

plt.ylabel("True Labels")
plt.xlabel("Predicted Labels")
plt.show()

In [ ]:
import random
correct_idxs = [i for i in range(len(true_labels)) if predicted_labels[i] == true_labels[i]]
incorrect_idxs = [i for i in range(len(true_labels)) if predicted_labels[i] != true_labels[i]]

sample_correct = random.sample(correct_idxs, min(4, len(correct_idxs)))
sample_incorrect = random.sample(incorrect_idxs, min(4, len(incorrect_idxs)))

fig, axes = plt.subplots(2, 4, figsize=(12, 6))

for i, idx in enumerate(sample_correct):
    img = test_dataset[idx]["pixel_values"].permute(1, 2, 0).numpy()
    axes[0, i].imshow(img)
    axes[0, i].set_title(f"✅ True: {id2label[true_labels[idx]]}\nPredicted: {id2label[predicted_labels[idx]]}")
    axes[0, i].axis("off")

for i, idx in enumerate(sample_incorrect):
    img = test_dataset[idx]["pixel_values"].permute(1, 2, 0).numpy()
    axes[1, i].imshow(img)
    axes[1, i].set_title(f"❌ True: {id2label[true_labels[idx]]}\nPredicted: {id2label[predicted_labels[idx]]}")
    axes[1, i].axis("off")

plt.show()


#Step7: Model Optimization

In [ ]:
# @title Hyperparameters
training_args = TrainingArguments(
    output_dir="tuned_model",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,                     # Learning rate
    lr_scheduler_type="cosine",
    per_device_train_batch_size=16,         # Batch size train
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,          # Batch size eval
    num_train_epochs=30,                     # Number of epochs
    warmup_ratio=0.1,
    logging_steps=3,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    processing_class=image_processor
)

trainer.train()